# Train Final Model

In [1]:
import numpy as np
import pandas as pd
import joblib
# sklearn import
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold


# my module imports
from optimalcodon.projects.rnastability.dataprocessing import get_data, general_preprocesing_pipeline


In [2]:
(train_x, train_y), (test_x, test_y) = get_data('../data/191004-TrainAndTestSets/')
print("{} points for training and {} for testing with {} features".format(
    train_x.shape[0], test_x.shape[0], test_x.shape[1]))


67775 points for training and 7576 for testing with 6 features


In [7]:
# TODO: change for the new trained model
# we load this to get the params
lasso = joblib.load("results-data/best-model-gridserach.joblib")


In [8]:
model = Pipeline([
    ('general', general_preprocesing_pipeline(train_x)), # see the code for general_preprocesing_pipeline
    ('polyfeaturs', PolynomialFeatures(degree=2)),
    ('zerovar', VarianceThreshold(threshold=0.0)),
    ('scaling', StandardScaler()),
    ('lasso', Lasso(**lasso.get_params()))
])

In [9]:
model.fit(train_x, train_y)

Pipeline(memory=None,
     steps=[('general', Pipeline(memory=None,
     steps=[('specific feature pipe', ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('codons', Pipeline(memory=None,
     steps=[('codon composition', CodonCompositionTransform...e=False, precompute=False,
   random_state=None, selection='cyclic', tol=0.0001, warm_start=False))])

In [10]:
## save trained model
joblib.dump(model, "results-data/final_model.joblib")

['results-data/final_model.joblib']

In [11]:
## predict test data
model.predict(test_x)

array([ 0.17833248, -0.52434314,  0.04860358, ...,  0.09396185,
        0.27326203,  0.02784497])

In [12]:
test_x['predicted'] = model.predict(test_x)
test_x['observed'] = test_y.values
(
    test_x
    .loc[:, ['specie', 'cell_type', 'datatype', 'predicted', 'observed']]
    .to_csv("results-data/test_data_predictions.csv")
)